In [2]:
#Import packages and secrets
import sys
import requests
import pandas as pd
import json
from datetime import datetime
from config import api_key_secret

#Set API call targets
#API calls limited to 500 a day, need to iterate through API call targets day by day
#ToDo: Get start and stop from a logging table
startiterating = 1
stopiterating = startiterating + 500

#Create empty list of lists to capture details for each successful loop run
looprunlog = []

#import apicalltargetid, state, suburb and postcodes into dataframe
df = pd.read_csv('suburbdatatest.csv')
df = df.loc[df["apicalltargetid"].between(startiterating,stopiterating)] #filter by apicalltargetid

#Convert dataframe into lists
apicalltargetids = ((df["apicalltargetid"]).astype(str).tolist())  #convert df to string and then convert to list
states = (df["state"]).tolist()
suburbs = (df["suburb"]).tolist()
postcodes = ((df["postcode"]).astype(str).tolist()) #convert df to string and then convert to list
apitypes = (df["apitype"]).tolist()

#Other URL Parameters
baseurl = "https://api.domain.com.au"
version = "v2"
apisubtypes = "Rent"
year = "2016"

#Authentication
header = {"X-API-Key" : api_key_secret}

#Loop through each item in lists, construct request URL, dump json response to directory 
for apicalltargetid, state, suburb, postcode, apitype, apisubtype in zip(apicalltargetids, states, suburbs, postcodes, apitypes, apisubtypes):    
    looprundatetime = datetime.now()
    fullurl = baseurl + "/" + version + "/" + apitype + "/" + str(state) + "/" + str(suburb) + "/" + str(postcode) + "?types=" + str(apisubtype)  + "=" + "&year" + str(year)
    response = requests.get(fullurl, headers=header)
    response = response.json()
    out_file = open(apitype+"_"+state+"_"+suburb+"_"+"_"+postcode+"_"+apisubtypes+"_"+year, "w")
    json.dump(response, out_file, indent = 6)
    out_file.close()
    looprunlog.append([apicalltargetid, state, suburb, postcode, apitype, apisubtypes, looprundatetime]) #append each loop to a log list of lists
    print(fullurl)

#Get looprunlog results in a dataframe
looprunlogdf = pd.DataFrame(looprunlog, columns = ['apicalltargetid','state','suburb','postcode','apitype','apisubtype','looprundatetime'])
print(looprunlogdf)



https://api.domain.com.au/v2/demographics/NSW/Pyrmont/2009?types=R=&year2016
https://api.domain.com.au/v2/demographics/WA/Willetton/6155?types=e=&year2016
  apicalltargetid state     suburb postcode       apitype apisubtype  \
0               1   NSW    Pyrmont     2009  demographics       Rent   
1               2    WA  Willetton     6155  demographics       Rent   

             looprundatetime  
0 2022-08-20 22:30:15.867920  
1 2022-08-20 22:30:16.502231  
